In [1]:
import pandas as pd
import os


DIRECTORY = '../data/'
for root, dirs, files in os.walk(DIRECTORY):
    file_names = files

In [2]:
# data is clean, primary key seems to be always in the very first column. Let's run a sanity check.
print('Number of nulls in the primary key_column\n')
for file in file_names:
    fpath = DIRECTORY + file
    df = pd.read_csv(fpath)

    print(f'File {file}, column {df.columns[0]}: {df.iloc[:, 0].isnull().sum()}')
    print(f'{len(df.iloc[:, 0].unique())} unique records in {len(df)} rows.')
    # the IDs will need to be autoincrements, let's definie the autoincrement start
    print(f'Max ID value: {df.iloc[:, 0].max()} \n')

Number of nulls in the primary key_column

File appearances.csv, column appearance_id: 0
1166215 unique records in 1166215 rows.
Max ID value: 4055287_958598 

File clubs.csv, column club_id: 0
411 unique records in 411 rows.
Max ID value: 83678 

File club_games.csv, column game_id: 0
61538 unique records in 123076 rows.
Max ID value: 4055287 

File competitions.csv, column competition_id: 0
43 unique records in 43 rows.
Max ID value: USC 

File games.csv, column game_id: 0
61538 unique records in 61538 rows.
Max ID value: 4055287 

File game_events.csv, column game_id: 0
61411 unique records in 559962 rows.
Max ID value: 4055287 

File players.csv, column player_id: 0
28459 unique records in 28459 rows.
Max ID value: 1134218 

File player_valuations.csv, column player_id: 0
27374 unique records in 424323 rows.
Max ID value: 1112706 



The assumptions I had are not 100% correct. I will not generate any default values for keys as all this data seems to be comming from a different database so I would expect the users to add a new data with the newly generated keys. Otherwise I will use autoincrement for most of the tables in this data set. 

In terms of the particular files, I see the following issues that I should be taken into consideration:

- in appearances.csv appearance_id is string, it looks like it's not used for any relation. Given it has a specific format I will not null constraint and this way I will force users to add that (this will be my approach for all tables btw).
- in clubs.csv club_id is correct key but it's not incrementing by 1. I'm not able to override that as club_id is used for multiple relations. Transfermarkt stores information about thousands of clubs so most likely they use the same approach but the data set contains only 400 clubs from top leagues.
- in club_games.csv all game_id needs to be combined with club_id to have a unique primary key
- in competitions.csv the key is string, need to change table ddl. I will not set any default value and 'not null' condition will force db users to provide this key.
- in game_events.csv, and player_valuations.csv, similar to club_games we need to create another columns with artificially created key if we want to have that unique. Normally I would go with MD5 but here for the simplicity I will keep going with a simple concat.